# The Price is Right
Today we build a more complex solution for estimating prices of goods.

This notebook: create a RAG database with our 400,000 training data
preperation for  2D






In [ ]:
# This cell loads all the Python libraries we need — data stuff, embedding model, Chroma, plotting. If any import fails, install that package in myenv

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [ ]:
# Load secrets from .env (OpenAI and HF tokens) and set a local DB folder name used by Chroma

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore"

In [33]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
# Another import after Logging in to Hugging Face

from items import Item

In [ ]:
# Load the pre-saved training data (train.pkl) that contains all product items. This is the dataset we’ll vectorize

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

In [ ]:
# Just showing the first item’s prompt so you can see data format and confirm load worked
train[0].prompt

'How much does this cost to the nearest dollar?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7\n\nPrice is $227.00'

# Now create a Chroma Datastore
we created a Chroma datastore with 123 documents representing chunks of objects from our fictional company Insurellm.

Now we will create a Chroma datastore with 400,000 products from our training dataset!

Note that we won't be using LangChain, but the API is very straightforward and consistent with before.



In [ ]:
# We create a persistent Chroma client that uses DB = "products_vectorstore" to store vectors on disk
client = chromadb.PersistentClient(path=DB)

In [ ]:
# Check if a collection named "products" exists. If yes, delete it (so we start clean). Then create a new "products" collection
collection_name = "products"

# Get list of existing collections (names only)
existing_collections = [col.name for col in client.list_collections()]

# Delete if it already exists
if collection_name in existing_collections:
    client.delete_collection(name=collection_name)
    print(f"Deleted existing collection: {collection_name}")

# Now create a new one
collection = client.create_collection(name=collection_name)
print(f"Created new collection: {collection_name}")


Deleted existing collection: products
Created new collection: products


# Introducing the SentenceTransfomer
The all-MiniLM is a very useful model from HuggingFace that maps sentences & paragraphs to a 384 dimensional dense vector space and is ideal for tasks like semantic search.


https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


It can run pretty quickly locally.

Last time we used OpenAI embeddings to produce vector embeddings. Benefits compared to OpenAI embeddings:

It's free and fast!
We can run it locally, so the data never leaves our box - might be useful if you're building a personal RAG

In [ ]:
# Instantiate the sentence-transformers/all-MiniLM-L6-v2 model. This maps text to 384-d vectors
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# Quick test: encode a sample sentence to verify model works and returns a vector

vector = model.encode(["Well hi there"])[0]
vector

array([-9.46715921e-02,  4.27619070e-02,  5.51620349e-02, -5.10994985e-04,
        1.16203846e-02, -6.80130348e-02,  2.76405867e-02,  6.06974773e-02,
        2.88530439e-02, -1.74128152e-02, -4.94346730e-02,  2.30992734e-02,
       -1.28614930e-02, -4.31402549e-02,  2.17509773e-02,  4.26549017e-02,
        5.10500222e-02, -7.79727325e-02, -1.23247214e-01,  3.67456153e-02,
        4.54111397e-03,  9.47937891e-02, -5.53099439e-02,  1.70641243e-02,
       -2.92872153e-02, -4.47124727e-02,  2.06784774e-02,  6.39319867e-02,
        2.27427743e-02,  4.87789139e-02, -2.33504479e-03,  4.72859293e-02,
       -2.86259428e-02,  2.30624359e-02,  2.45130248e-02,  3.95681821e-02,
       -4.33176607e-02, -1.02316648e-01,  2.79879849e-03,  2.39304397e-02,
        1.61556248e-02, -8.99083726e-03,  2.07255650e-02,  6.40122816e-02,
        6.89179078e-02, -6.98361024e-02,  2.89763510e-03, -8.10989141e-02,
        1.71122495e-02,  2.50652479e-03, -1.06529027e-01, -4.87732477e-02,
       -1.67762004e-02, -

In [ ]:
# Helper functions to compute cosine similarity between two texts — handy for quick semantic checks
import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2])
    similarity = cosine_similarity(vector1, vector2)
    print(f"Similarity between {text1} and {text2} is {similarity*100:.1f}%")

In [ ]:
# We test similarity: Java vs C++, Java vs mug, "Cup of Java" vs mug. Shows model senses context

how_similar("Java", "C++")
how_similar("Java", "mug")
how_similar("Cup of Java", "mug")

Similarity between Java and C++ is 50.7%
Similarity between Java and mug is 25.8%
Similarity between Cup of Java and mug is 49.3%


In [ ]:
# This function strips the question wrapper from each item and returns the plain product description — the exact text we’ll embed

def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [ ]:
# Quick check to ensure description() works and shows a clean description
description(train[0])

'Delphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7'

# Now we populate our RAG datastore
The next cell populates the 400,000 items in Chroma.

we will go with a can change to:
NUMBER_OF_DOCUMENTS = 30000
And that's plenty for a perfectly good RAG pipeline.

Just note that if you interrupt the below cell while it's running, you might need to clear out the Chroma datastore (by rerunning the earlier cell that deletes the collection), before you run it again. Otherwise it will complain that there are existing documents with the same ID.

In [ ]:
# NUMBER_OF_DOCUMENTS = len(train)

#we will vectorize and add 400k items to Chroma, the system is too slow so we will only take 30000
NUMBER_OF_DOCUMENTS = 30000

for i in tqdm(range(0, NUMBER_OF_DOCUMENTS, 1000)):
    documents = [description(item) for item in train[i: i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+len(documents))]
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

100%|██████████| 30/30 [10:19<00:00, 20.65s/it]


In [19]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [20]:
# It is very fun turning this up to 400_000 and seeing the full dataset visualized,
# but it almost crashes my box every time so do that at your own risk!! 10_000 is safe!

MAXIMUM_DATAPOINTS = 30_000

In [21]:
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)

In [22]:
collection = client.get_or_create_collection('products')

In [23]:
CATEGORIES = ['Appliances', 'Automotive', 'Cell_Phones_and_Accessories', 'Electronics','Musical_Instruments', 'Office_Products', 'Tools_and_Home_Improvement', 'Toys_and_Games']
COLORS = ['red', 'blue', 'brown', 'orange', 'yellow', 'green' , 'purple', 'cyan']

In [24]:
# Prework
result = collection.get(include=['embeddings', 'documents', 'metadatas'], limit=MAXIMUM_DATAPOINTS)
vectors = np.array(result['embeddings'])
documents = result['documents']
categories = [metadata['category'] for metadata in result['metadatas']]
colors = [COLORS[CATEGORIES.index(c)] for c in categories]

In [25]:
# Let's try a 2D chart

tsne = TSNE(n_components=2, random_state=42, n_jobs=-1)
reduced_vectors = tsne.fit_transform(vectors)

In [30]:

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=3, color=colors, opacity=0.7),
)])

fig.update_layout(
    title='2D Chroma Vectorstore Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed